### Modeling
- input data
    - keyword(조명, 외장하드, 블루투스 스피커, 물티슈)
    * 쿠팡랭킹 top36까지 크롤링 / top36까지 총 댓글 수 확인
- 전처리
    - 'pos_filtered' == 0인 데이터 삭제
    - filtering 품사 줄이고 stop word 수동 설정

In [1]:
def make_df_train(keyword, item_count_start=0, item_count_end=36):
    import requests
    from scrapy.http import TextResponse
    import json
    search_url = "http://www.coupang.com/np/search?component=&q={}".format(keyword)
    rep = requests.get(search_url)
    response = TextResponse(rep.url, body=rep.text, encoding='utf-8')

    # 상품마다의 링크를 불러오기에 앞서 아이템 Top36 리스트 불러오기
    products = json.loads(response.xpath('//*[@id="productList"]/@data-products').extract()[0])['indexes'][item_count_start:item_count_end]

    df = pd.DataFrame(columns=['rating', "item_nbr", "item", "option", "date", "name", "re_title", 'review']) 

    top36_review_total_cnt = 0
    
    for product in products:
        
        # Top36 상품마다의 상품평 개수(나중에 불러오는 정보라 상품링크 안에서는 xpath로 바로 가져올 수 없음)
        try:
            review_count = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
            top36_review_total_cnt += review_count
            
        except Exception as e:
            print('No reviews')
        
        # product for문 안에서 수행하는 코드    
        def make_url(product, review_count):
            page_count = (review_count//100)+1
            review_urls = []
            for page in range(1, page_count+1):
                review_urls.append('http://www.coupang.com/vp/product/reviews?productId={}&page={}&size=100&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=2&ratingSummary=true'.format(product, page))
                # size는 100이 최대로 설정되어 있음
            return review_urls


        # Top36 상품마다의 크롤링할 url(리뷰 크롤링은 '상품평 보기' 클릭 후 나타나는 reviews Request URL에서 해야함)
#         review_urls[product] = make_url(product, review_count)
        review_url = make_url(product, review_count) # 리스트로 반환됨

        for url in review_url:

            review_url = url
            review_rep = requests.get(review_url)
            review_response = TextResponse(review_rep.url, body=review_rep.text, encoding='utf-8')        

            # Top36 상품마다의 상품평 개수
            review_total_cnt = int(review_response.xpath('/html/body/div[2]/@data-review-total-count').extract()[0])

            # Top36 상품마다 한 page의 상품평 개수
            review_url_cnt = int(float(review_response.xpath('count(/html/body/article)').extract()[0]))

            if review_url_cnt < 100:          
                for i in range(1, review_url_cnt+1): # 한개의 review_url에서 100개를 훑고 그 다음 review_url로 넘어가면 됨
                    item_nbr = product

                    date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                    name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                    item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                    if len(item_option_ls) == 1:
                        item = item_option_ls[0]

                    else:    
                        item = item_option_ls[0]
                        option = item_option_ls[1]

                    if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                        re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                        review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                    else:
                        re_title = ['.']
                        review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                    rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                    df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}

            else:
                for i in range(1, 101):
                    item_nbr = product

                    date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                    name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                    item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                    if len(item_option_ls) == 1:
                        item = item_option_ls[0]

                    else:    
                        item = item_option_ls[0]
                        option = item_option_ls[1]

                    if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                        re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                        review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                    else:
                        re_title = ['.']
                        review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                    rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                    df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}
                    
    def re_title_filter(a):
        for i in range(len(a[0].split('\n'))):
            if len(a[0].split('\n')[i].strip()) != 0:
                return a[0].split('\n')[i].strip()
            else:
                return '.'

    def review_filter(a):
        s = ''
        for i in range(len(a)):
            for ii in range(len(a[i].split('\n'))):
                if a[i].split('\n')[ii].strip() != 0:
                    s += a[i].split('\n')[ii].strip() + ' '
        return s

    def kkma_pos(a):
        from konlpy.tag import Kkma
        kkma = Kkma()
        return kkma.pos(a)

    def kkma_pos_filter(a):
        """
        df['pos']를 받아 품사 필터링을 거쳐 해당 키워드 리스트 반환
        """
        pos_ls = ['NNG','NNP','VA','UN','XR','MAG','ECE']
        ls = []
        for i in range(len(a)):
            #NNG(보통 명사), NNP(고유 명사), VA(형용사), XR(어근), MAG(일반 부사), UN(명사추정범주)
            if a[i][1] in pos_ls:
                ls.append(a[i][0])
        return ls

    def rating_filter(a):
        if a == '3' or a == '4' or a == '5':
            return 1
            # 긍정
        else:
            return 0
            # 부정
    
    df['re_title_filtered'] = df['re_title'].apply(re_title_filter)
    df['review_filtered'] = df['review'].apply(review_filter)
    df['full_review'] = df['re_title_filtered']+df['review_filtered']
    df['pos'] = df['full_review'].apply(kkma_pos)
    df['pos_filtered'] = df['pos'].apply(kkma_pos_filter)
    df['rating_filtered'] = df['rating'].apply(rating_filter)
    
    # 'pos_filtered' == 0인 데이터 삭제
    idx = []
    for i in range(len(df)):
        if len(df['pos_filtered'][i]) == 0:
            idx.append(i)
    df = df.drop(index=idx).reset_index()
    
    print("{}'s Top36 Review Total Count : {}".format(keyword, top36_review_total_cnt))
    print("{}'s Top36 Filtered Review Count : {}".format(keyword, len(df)))
    
    df_fin = df[['rating', 'rating_filtered', 'item_nbr', 'item', 'option', 'date', 'name', 'full_review', 'pos', 'pos_filtered']]
    
    return df_fin

In [2]:
df_train = make_df_train('샘키즈')
df_train.tail()

No reviews
No reviews
No reviews
No reviews
No reviews
No reviews
No reviews
No reviews
No reviews
샘키즈's Top36 Review Total Count : 890
샘키즈's Top36 Filtered Review Count : 464


,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
459,5,1,46457136,[리바트 이즈마인] 프렌즈아이 마카롱 3x5 일반 수납장 슬림형 (그린),C형 (몸통)크림화이트/(박스)코랄민트(C),2017.02.19,김*경,. 수납 쨩이예요~ 아이에게도 정리하는 습관이 잘될거 같아요~ 빠른 배송 감쏴...,"[(., SF), (수납, NNG), (쨩이예, UN), (요, JX), (~, S...","[수납, 쨩이예, 아이, 정리, 습관, 같, 빠르, 배송, 감]"
460,5,1,46457136,[리바트 이즈마인] 프렌즈아이 마카롱 3x5 일반 수납장 슬림형 (그린),C형 (몸통)크림화이트/(박스)코랄민트(C),2017.07.27,김*태,. 상품평은첨써보내요.ㅋ 수납장도맘에들고.아기가열어도 서랍이 빠지지 않아서 안...,"[(., SF), (상품평, NNG), (은, JX), (첨, NNG), (쓰, V...","[상품평, 첨, ㅋ, 수납장, 맘, 고, 아기, 서랍, 안전, 고, 특히, 기사님,..."
461,5,1,46457265,[리바트 이즈마인] 프렌즈아이 마카롱 3x5 일반 수납장 슬림형 (화이트),C형 (몸통)크림화이트/(박스)코랄민트(C),2017.07.21,이*진,. 안전한 배송과 기사님 께서 꼼꼼히 해주셔서 너무 감사합니다 제품이 너무 짱짱...,"[(., SF), (안전, NNG), (하, XSV), (ㄴ, ETD), (배송, ...","[안전, 배송, 기사님, 꼼꼼히, 너무, 감사, 제품, 너무, 짱짱, 좋, 냄새, ..."
462,4,1,46411189,[리바트 이즈마인] 프렌즈아이 마카롱 2x5 일반 수납장 슬림형(옐로우),C형 (몸통)크림화이트/(박스)코랄민트(C),2017.02.14,윤*영,. 아이 장난감 여러 가지 물건들을 한번에 정리되어 너무 깔끔하고 좋습니다. 디...,"[(., SF), (아이, NNG), (장난감, NNG), (여러, MDT), (가...","[아이, 장난감, 물건, 한번, 정리, 너무, 깔끔, 고, 좋, 디자인, 색상, 맘..."
463,4,1,46358785,[리바트 이즈마인] 프렌즈아이 마카롱 1x5 일반 수납장 (화이트),C형 (몸통)크림화이트/(박스)코랄민트(C),2017.03.08,원*리,. 깔끔하고 이뻐요 수납하기가 용이하겠어요^^,"[(., SF), (깔끔, XR), (하, XSA), (고, ECE), (이쁘, V...","[깔끔, 고, 이쁘, 수납, 용이]"


In [ ]:
# df_train로 만든 단어장
# train 데이터와 test 데이터의 단어장은 동일해야한다

In [ ]:
# https://datascienceschool.net/view-notebook/3387abe1784248339e418835297e17d2/
# from konlpy.tag import Twitter
# pos_tagger = Twitter()

# def tokenize_pos(doc):
#     return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]
# model3 = Pipeline([
#             ('vect', CountVectorizer(tokenizer=tokenize_pos)), 
#             ('mb', MultinomialNB()),
#         ])

In [ ]:
def Vectorizer(df_train, pos_filtered_col, df_input):
    """
    df_fin의 'pos_filtered' column을 받아 vectorized df를 생성해주는 함수
    """
    train_corpus = []
    for i in range(len(df_train[pos_filtered_col])):
        for ii in range(len(df_train[pos_filtered_col][i])):
            train_corpus.append(df_train[pos_filtered_col][i][ii])
            
    from sklearn.feature_extraction.text import CountVectorizer
    vect = CountVectorizer(token_pattern=r"\b\w+\b") # 한 글자도 corpus에 포함될 수 있게 해주는 정규표현식
    vect.fit(train_corpus)

    vect_ls = []
    for i in range(len(df_input)):
        vect_ls.append((vect.transform(df_input[pos_filtered_col][i]).toarray().sum(axis=0).tolist()))
        # 원래 문자열 하나가 문장으로 간주되는데
    
    df_vec = pd.DataFrame(vect_ls)
    return df_vec

In [10]:
def Vectorizer(df_train, pos_filtered_col, df_input):
    """
    df_fin의 'pos_filtered' column을 받아 vectorized df를 생성해주는 함수
    """
    train_corpus = []
    for i in range(len(df_train[pos_filtered_col])):
        for ii in range(len(df_train[pos_filtered_col][i])):
            train_corpus.append(df_train[pos_filtered_col][i][ii])
            
    from sklearn.feature_extraction.text import CountVectorizer
    vect = CountVectorizer(token_pattern=r"\b\w+\b") # 한 글자도 corpus에 포함될 수 있게 해주는 정규표현식
    vect.fit(train_corpus)

    vect_ls = []
    for i in range(len(df_input)):
        vect_ls.append((vect.transform(df_input[pos_filtered_col][i]).toarray().sum(axis=0).tolist()))
        # 원래 문자열 하나가 문장으로 간주되는데
    
    df_vec = pd.DataFrame(vect_ls)
    return df_vec

In [11]:
%%time
X_train = Vectorizer(df_train, 'pos_filtered', df_train)
y_train = df_train['rating_filtered']

Wall time: 1.28 s


In [16]:
df_train['pos_filtered'][459]

'수납,쨩이예,아이,정리,습관,같,빠르,배송,감,'

In [12]:
df_train.tail()

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
459,5,1,46457136,[리바트 이즈마인] 프렌즈아이 마카롱 3x5 일반 수납장 슬림형 (그린),C형 (몸통)크림화이트/(박스)코랄민트(C),2017.02.19,김*경,. 수납 쨩이예요~ 아이에게도 정리하는 습관이 잘될거 같아요~ 빠른 배송 감쏴...,"[(., SF), (수납, NNG), (쨩이예, UN), (요, JX), (~, S...","[수납, 쨩이예, 아이, 정리, 습관, 같, 빠르, 배송, 감]"
460,5,1,46457136,[리바트 이즈마인] 프렌즈아이 마카롱 3x5 일반 수납장 슬림형 (그린),C형 (몸통)크림화이트/(박스)코랄민트(C),2017.07.27,김*태,. 상품평은첨써보내요.ㅋ 수납장도맘에들고.아기가열어도 서랍이 빠지지 않아서 안...,"[(., SF), (상품평, NNG), (은, JX), (첨, NNG), (쓰, V...","[상품평, 첨, ㅋ, 수납장, 맘, 고, 아기, 서랍, 안전, 고, 특히, 기사님,..."
461,5,1,46457265,[리바트 이즈마인] 프렌즈아이 마카롱 3x5 일반 수납장 슬림형 (화이트),C형 (몸통)크림화이트/(박스)코랄민트(C),2017.07.21,이*진,. 안전한 배송과 기사님 께서 꼼꼼히 해주셔서 너무 감사합니다 제품이 너무 짱짱...,"[(., SF), (안전, NNG), (하, XSV), (ㄴ, ETD), (배송, ...","[안전, 배송, 기사님, 꼼꼼히, 너무, 감사, 제품, 너무, 짱짱, 좋, 냄새, ..."
462,4,1,46411189,[리바트 이즈마인] 프렌즈아이 마카롱 2x5 일반 수납장 슬림형(옐로우),C형 (몸통)크림화이트/(박스)코랄민트(C),2017.02.14,윤*영,. 아이 장난감 여러 가지 물건들을 한번에 정리되어 너무 깔끔하고 좋습니다. 디...,"[(., SF), (아이, NNG), (장난감, NNG), (여러, MDT), (가...","[아이, 장난감, 물건, 한번, 정리, 너무, 깔끔, 고, 좋, 디자인, 색상, 맘..."
463,4,1,46358785,[리바트 이즈마인] 프렌즈아이 마카롱 1x5 일반 수납장 (화이트),C형 (몸통)크림화이트/(박스)코랄민트(C),2017.03.08,원*리,. 깔끔하고 이뻐요 수납하기가 용이하겠어요^^,"[(., SF), (깔끔, XR), (하, XSA), (고, ECE), (이쁘, V...","[깔끔, 고, 이쁘, 수납, 용이]"


In [9]:
X_train.tail()

,0,1,2,3,4,5,6,7,8,9,...,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435
459,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
460,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
461,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
462,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model = MultinomialNB()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(classification_report(y_train, y_train_pred))

             precision    recall  f1-score   support

          0       1.00      0.86      0.93        22
          1       0.99      1.00      1.00       442

avg / total       0.99      0.99      0.99       464



In [49]:
def make_df_test(link):
    import requests
    from scrapy.http import TextResponse
    import json
    
    input_url = link.split('?')[0]
    product = link.split('?')[0].split("/")[5]
    
    # 상품마다의 링크를 불러오기에 앞서 아이템 Top36 리스트 불러오기
#     products = json.loads(response.xpath('//*[@id="productList"]/@data-products').extract()[0])['indexes'][:36]

    df = pd.DataFrame(columns=['rating', "item_nbr", "item", "option", "date", "name", "re_title", 'review']) 

    reveiw_cnt_url = 'http://www.coupang.com/vp/product/reviews?productId={}&page=1&size=100&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=2&ratingSummary=true'.format(product)
    reveiw_cnt_rep = requests.get(reveiw_cnt_url)
    reveiw_cnt_response = TextResponse(reveiw_cnt_rep.url, body=reveiw_cnt_rep.text, encoding='utf-8')
    
    item_name = reveiw_cnt_response.xpath('/html/body/article[1]/div[1]/div[4]/text()').extract()[0].split(',')[0]
    review_total_cnt = int(reveiw_cnt_response.xpath('/html/body/div[2]/@data-review-total-count').extract()[0])

    def make_url(product, review_total_cnt):
        page_count = (review_total_cnt//100)+1
        review_urls = []
        for page in range(1, page_count+1):
            review_urls.append('http://www.coupang.com/vp/product/reviews?productId={}&page={}&size=100&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=2&ratingSummary=true'.format(product, page))
            # size는 100이 최대로 설정되어 있음
        return review_urls

    review_url = make_url(product, review_total_cnt) # 리스트로 반환됨

    for url in review_url:

        review_url = url
        review_rep = requests.get(review_url)
        review_response = TextResponse(review_rep.url, body=review_rep.text, encoding='utf-8')        

        # Top36 상품마다의 상품평 개수
        review_total_cnt = int(review_response.xpath('/html/body/div[2]/@data-review-total-count').extract()[0])

        # Top36 상품마다 한 page의 상품평 개수
        review_url_cnt = int(float(review_response.xpath('count(/html/body/article)').extract()[0]))

        if review_url_cnt < 100:          
            for i in range(1, review_url_cnt+1): # 한개의 review_url에서 100개를 훑고 그 다음 review_url로 넘어가면 됨
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}

        else:
            for i in range(1, 101):
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}
                    
    def re_title_filter(a):
        for i in range(len(a[0].split('\n'))):
            if len(a[0].split('\n')[i].strip()) != 0:
                return a[0].split('\n')[i].strip()
            else:
                return '.'

    def review_filter(a):
        s = ''
        for i in range(len(a)):
            for ii in range(len(a[i].split('\n'))):
                if a[i].split('\n')[ii].strip() != 0:
                    s += a[i].split('\n')[ii].strip() + ' '
        return s

    def kkma_pos(a):
        from konlpy.tag import Kkma
        kkma = Kkma()
        return kkma.pos(a)

    def kkma_pos_filter(a):
        """
        df['pos']를 받아 품사 필터링을 거쳐 해당 키워드 리스트 반환
        """
        pos_ls = ['NNG','NNP','VA','UN','XR','MAG','ECE']
        ls = []
        for i in range(len(a)):
            #NNG(보통 명사), NNP(고유 명사), VA(형용사), XR(어근), MAG(일반 부사), UN(명사추정범주)
            if a[i][1] in pos_ls:
                ls.append(a[i][0])
        return ls

    def rating_filter(a):
        if a == '3' or a == '4' or a == '5':
            return 1
            # 긍정
        else:
            return 0
            # 부정
    
    df['re_title_filtered'] = df['re_title'].apply(re_title_filter)
    df['review_filtered'] = df['review'].apply(review_filter)
    df['full_review'] = df['re_title_filtered']+df['review_filtered']
    df['pos'] = df['full_review'].apply(kkma_pos)
    df['pos_filtered'] = df['pos'].apply(kkma_pos_filter)
    df['rating_filtered'] = df['rating'].apply(rating_filter)
    
    # 'pos_filtered' == 0인 데이터 삭제
    idx = []
    for i in range(len(df)):
        if len(df['pos_filtered'][i]) == 0:
            idx.append(i)
    df = df.drop(index=idx).reset_index()
    
    print("{}'s Review Total Count : {}".format(item_name, review_total_cnt))
    print("{}'s Filtered Review Count : {}".format(item_name, len(df)))
    
    df_fin = df[['rating', 'rating_filtered', 'item_nbr', 'item', 'option', 'date', 'name', 'full_review', 'pos', 'pos_filtered']]
    
    return df_fin

In [50]:
df_test = make_df_test('http://www.coupang.com/vp/products/30366312?itemId=115656842&vendorItemId=3231901332&q=%EC%83%98%ED%82%A4%EC%A6%88&itemsCount=36&searchId=18bfe53bfd5843c88b73e3b0e4a31737&rank=1')
df_test.tail()

한샘 샘키즈 수납장 1305 블라썸 민트's Review Total Count : 395
한샘 샘키즈 수납장 1305 블라썸 민트's Filtered Review Count : 204


,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
199,5,1,30366312,한샘 샘키즈 수납장 1305 블라썸 민트,A형:(몸통)크림화이트/(박스)민트그레이(A),2017.10.31,베르디아나,. 깔끔하니 예뻐요,"[(., SF), (깔끔, XR), (하, XSA), (니, ECD), (예쁘, V...","[깔끔, 예쁘]"
200,5,1,30366312,한샘 샘키즈 수납장 1305 블라썸 민트,A형:(몸통)크림화이트/(박스)민트그레이(A),2017.10.26,성*현,. 튼튼하고 수납도많이되고 좋아요,"[(., SF), (튼튼, NNP), (하, XSV), (고, ECE), (수납, ...","[튼튼, 고, 수납, 많이, 고, 좋]"
201,4,1,30366312,한샘 샘키즈 수납장 1305 블라썸 민트,B형:(몸통)크림화이트/(박스)하프민트(B),2017.10.22,이*현,. 간편하고 깔끔하니 괜찮아요,"[(., SF), (간편, NNG), (하, XSV), (고, ECE), (깔끔, ...","[간편, 고, 깔끔, 괜찮]"
202,5,1,30366312,한샘 샘키즈 수납장 1305 블라썸 민트,A형:(몸통)크림화이트/(박스)민트그레이(A),2017.10.16,권*진,. 너무 좋아요 깔끔하고ㅋ,"[(., SF), (너무, MAG), (좋, VA), (아요, EFN), (깔끔, ...","[너무, 좋, 깔끔, 고, ㅋ]"
203,5,1,30366312,한샘 샘키즈 수납장 1305 블라썸 민트,E형:(몸통)메이플/(박스)하프민트(E),2017.09.27,HYMom,. 애기 옷이랑 장난감수납에 좋네요,"[(., SF), (애기, NNG), (옷, NNG), (이랑, JC), (장난감,...","[애기, 옷, 장난감, 수납, 좋]"


In [51]:
X_test = Vectorizer(df_train, 'pos_filtered', df_test)
y_test = df_test['rating_filtered']

In [52]:
X_test.tail()

,0,1,2,3,4,5,6,7,8,9,...,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435
199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
200,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
203,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
# model = MultinomialNB()
# model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          0       1.00      0.80      0.89         5
          1       0.99      1.00      1.00       199

avg / total       1.00      1.00      0.99       204



------------------------------------------------------

In [5]:
def review_cnt_check(keyword, item_count_start=0, item_count_end=36):
    import requests
    from scrapy.http import TextResponse
    import json
    search_url = "http://www.coupang.com/np/search?component=&q={}".format(keyword)
    rep = requests.get(search_url)
    response = TextResponse(rep.url, body=rep.text, encoding='utf-8')

    # 상품마다의 링크를 불러오기에 앞서 아이템 Top36 리스트 불러오기
    products = json.loads(response.xpath('//*[@id="productList"]/@data-products').extract()[0])['indexes'][item_count_start:item_count_end]
    
    review_total_cnt = 0
    
    for idx, product in enumerate(products):
        
        # Top36 상품마다의 상품평 개수(나중에 불러오는 정보라 상품링크 안에서는 xpath로 바로 가져올 수 없음)
        try:
            review_count = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
            review_total_cnt += review_count
            print('{}번째 item {}의 review 수 : {}'.format(idx, product, review_count))
            
        except Exception as e:
            print('No reviews')
            
    print('{}번째부터 {}번째 item의 총 review 수 : {}'.format(item_count_start, item_count_end-1, review_total_cnt))

In [6]:
review_cnt_check('조명')

0번째 item 1001428의 review 수 : 4857
1번째 item 36815959의 review 수 : 645
2번째 item 55254805의 review 수 : 15
3번째 item 35863481의 review 수 : 129
4번째 item 36815948의 review 수 : 151
5번째 item 1001428의 review 수 : 4857
6번째 item 36551328의 review 수 : 124
7번째 item 36234235의 review 수 : 684
8번째 item 22678653의 review 수 : 215
9번째 item 16131155의 review 수 : 32
10번째 item 1001428의 review 수 : 4857
11번째 item 27470080의 review 수 : 1437
12번째 item 47836012의 review 수 : 89
13번째 item 4915201의 review 수 : 295
14번째 item 56837841의 review 수 : 5
15번째 item 1001428의 review 수 : 4857
16번째 item 65647114의 review 수 : 94
17번째 item 36234235의 review 수 : 684
18번째 item 23593043의 review 수 : 17
19번째 item 36815959의 review 수 : 645
20번째 item 47836013의 review 수 : 76
21번째 item 44920528의 review 수 : 14
22번째 item 8341176의 review 수 : 390
23번째 item 18042509의 review 수 : 20
24번째 item 36815955의 review 수 : 58
25번째 item 11603277의 review 수 : 83
26번째 item 26447051의 review 수 : 10
27번째 item 34096012의 review 수 : 4
28번째 item 18042509의 review 수 : 20
29번째 item 10

In [54]:
df_train = make_df_train('조명')
df_train.tail()

조명's Top36 Review Total Count : 25905
조명's Top36 Filtered Review Count : 13888


,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
13883,5,1,19057548,에브리띵어스 리모컨 LED 글라스돔 무드등/수면등/수유등,원통 (블랙),2017.12.17,쿠마리,. 아쥬 좋음 리모컨 굿,"[(., SF), (아, VV), (아, ECS), (쥬, UN), (좋, VA),...","[쥬, 좋, 리모컨, 굿]"
13884,5,1,19057548,에브리띵어스 리모컨 LED 글라스돔 무드등/수면등/수유등,원통 (블랙),2017.12.08,김*애,. 엄청 이쁘네용ㅎㅎ 완전 추천입니다!!,"[(., SF), (엄청, MAG), (이쁘, VA), (네요, EFN), (ㅎㅎ,...","[엄청, 이쁘, 완전, 추천]"
13885,1,0,19057548,에브리띵어스 리모컨 LED 글라스돔 무드등/수면등/수유등,원통 (블랙),2017.11.27,박*숙,. 건전지가 금방ᆢㅠ,"[(., SF), (건전지, NNG), (가, JKS), (금방, MAG), (ᆢ,...","[건전지, 금방, ㅠ]"
13886,5,1,19057548,에브리띵어스 리모컨 LED 글라스돔 무드등/수면등/수유등,원통 (블랙),2017.11.20,홍*희,. 친구 선물,"[(., SF), (친구, NNG), (선물, NNG)]","[친구, 선물]"
13887,5,1,19057548,에브리띵어스 리모컨 LED 글라스돔 무드등/수면등/수유등,원통 (블랙),2017.11.03,이*재,. 가격대비 정말좋아요,"[(., SF), (가격대비, NNP), (정말, MAG), (좋, VA), (아요...","[가격대비, 정말, 좋]"


In [55]:
X_train = Vectorizer(df_train, 'pos_filtered', df_train)
y_train = df_train['rating_filtered']

In [56]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model = MultinomialNB()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(classification_report(y_train, y_train_pred))

             precision    recall  f1-score   support

          0       0.79      0.70      0.74       798
          1       0.98      0.99      0.99     13090

avg / total       0.97      0.97      0.97     13888



In [58]:
df_test = make_df_test('http://www.coupang.com/vp/products/1001428?itemId=4200574&vendorItemId=3005056040&q=%EC%A1%B0%EB%AA%85&itemsCount=36&searchId=fa72967012e141a3b27088f5e900b61b&rank=0')
df_test.tail()

스피아노 스프링 플로어 장스탠드 SS-2000A's Review Total Count : 4853
스피아노 스프링 플로어 장스탠드 SS-2000A's Filtered Review Count : 2621


,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
2616,4,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,화이트,2015.12.18,김*영,. 램프를 따로 사야 하는 것빼면 가격대비 좋네요!,"[(., SF), (램프, NNG), (를, JKO), (따로, MAG), (사, ...","[램프, 따로, 면, 가격, 대비, 좋]"
2617,5,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,화이트,2015.12.15,권*출,. 강추,"[(., SF), (강추, NNG)]",[강추]
2618,3,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,화이트,2015.12.09,안*권,. 좋네요,"[(., SF), (좋, VA), (네요, EFN)]",[좋]
2619,3,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2015.12.08,이*재,. 좋은데 튼튼했으며 더좋을듯..,"[(., SF), (좋, VA), (은데, ECD), (튼튼, NNP), (하, X...","[좋, 튼튼, 으며, 더, 좋]"
2620,3,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2015.11.16,김*얀,. 잘도착했어요~,"[(., SF), (잘, MAG), (도착, NNG), (하, XSV), (었, E...","[잘, 도착]"


In [59]:
X_test = Vectorizer(df_train, 'pos_filtered', df_test)
y_test = df_test['rating_filtered']

In [60]:
# model = MultinomialNB()
# model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          0       0.83      0.73      0.78       131
          1       0.99      0.99      0.99      2490

avg / total       0.98      0.98      0.98      2621



In [96]:
review_cnt_check('외장하드', 0, 26)

1번째 item 25271683의 review 수 : 565
2번째 item 4889527의 review 수 : 3023
3번째 item 88785654의 review 수 : 36
4번째 item 60867504의 review 수 : 78
5번째 item 49867193의 review 수 : 259
6번째 item 14820704의 review 수 : 1475
7번째 item 1148353의 review 수 : 2401
8번째 item 57040999의 review 수 : 202
9번째 item 87782199의 review 수 : 55
10번째 item 49867193의 review 수 : 259
11번째 item 14820704의 review 수 : 1475
12번째 item 1148353의 review 수 : 2401
13번째 item 1148365의 review 수 : 1527
14번째 item 87782199의 review 수 : 55
15번째 item 49867193의 review 수 : 259
16번째 item 14820708의 review 수 : 332
17번째 item 1148353의 review 수 : 2401
18번째 item 1148365의 review 수 : 1527
19번째 item 4956527의 review 수 : 1765
20번째 item 14820704의 review 수 : 1475
21번째 item 25271665의 review 수 : 687
22번째 item 1148353의 review 수 : 2401
23번째 item 1148365의 review 수 : 1527
24번째 item 4956527의 review 수 : 1765
25번째 item 14820704의 review 수 : 1475
26번째 item 106697980의 review 수 : 6
1번째부터 26번째 item의 총 review 수 : 29431


In [98]:
%%time
df_train = make_df_train('외장하드', 0, 26)
df_train.tail()

외장하드's Top36 Review Total Count : 29431
외장하드's Top36 Filtered Review Count : 12295
Wall time: 15min 38s


In [99]:
%%time
X_train = Vectorizer(df_train, 'pos_filtered', df_train)
y_train = df_train['rating_filtered']

Wall time: 33.6 s


In [100]:
df_train.tail()

,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
12290,5,1,106697980,엠지텍 테란 엣지 케이블 일체형 외장하드 Terran3.1 PMR,1TB,2018.08.23,최*환,. 외장하드 일체형은 처음 사용해보네요 케이블이 망가지면 불편하겠다.. 라는 생...,"[(., SF), (외장, NNG), (하드, NNG), (일체형, NNG), (은...","[외장, 하드, 일체형, 처음, 사용, 케이블, 면, 불편, 생각, 면서, 성비, ..."
12291,5,1,106697980,엠지텍 테란 엣지 케이블 일체형 외장하드 Terran3.1 PMR,1TB,2018.08.23,김*곤,. 역시 배송은 로켓배송 ㅋ 진짜빠르네요~ 받마마자 연결해서 쓰는데 연결하자마자...,"[(., SF), (역시, MAG), (배송, NNG), (은, JX), (로켓, ...","[역시, 배송, 로켓, 배송, ㅋ, 진짜, 빠르, 마자, 연결, 연결, 자마자, 바..."
12292,5,1,106697980,엠지텍 테란 엣지 케이블 일체형 외장하드 Terran3.1 PMR,1TB,2018.08.23,김*진,. 빠른배송 감사합니다. 케이스가 굉장히 견고하고 튼튼합니다! 떨어뜨리면 안되...,"[(., SF), (빠르, VA), (ㄴ, ETD), (배송, NNG), (감사, ...","[빠르, 배송, 감사, 케이스, 굉장히, 견고, 고, 튼튼, 면, 안되, 지만, 끄..."
12293,5,1,106697980,엠지텍 테란 엣지 케이블 일체형 외장하드 Terran3.1 PMR,1TB,2018.08.23,김*수,. 23시30분에 주문했는데도 다음날 바로 오네요 ! 로켓배송 기사님들 정말 수...,"[(., SF), (23, NR), (시, NNM), (30, NR), (분, NN...","[주문, 다음날, 바로, 로켓, 배송, 기사님, 정말, 수고, 많, 다, 그리고, ..."
12294,4,1,106697980,엠지텍 테란 엣지 케이블 일체형 외장하드 Terran3.1 PMR,1TB,2018.08.16,엄*식,. 좋아요 가볍고 선도 일체형이고 다만 뭐가 조잡한 느낌이 듭니다 ㅎㅎ,"[(., SF), (좋, VA), (아요, EFN), (가볍, VA), (고, EC...","[좋, 가볍, 고, 선도, 일체형, 고, 다만, 조잡, 느낌]"


In [101]:
X_train.tail()

,0,1,2,3,4,5,6,7,8,9,...,4410,4411,4412,4413,4414,4415,4416,4417,4418,4419
12290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12293,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model = MultinomialNB()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(classification_report(y_train, y_train_pred))

             precision    recall  f1-score   support

          0       0.81      0.82      0.82       905
          1       0.99      0.98      0.99     11390

avg / total       0.97      0.97      0.97     12295



In [111]:
df_test = make_df_test('http://www.coupang.com/vp/products/1168550?itemId=5040336&vendorItemId=3006743611&q=%EC%99%B8%EC%9E%A5%ED%95%98%EB%93%9C&itemsCount=36&searchId=4b1fbe311eaf480884ffe815a7143a12&rank=36')
df_test.tail()

삼성전자 J3 Portable 외장하드's Review Total Count : 626
삼성전자 J3 Portable 외장하드's Filtered Review Count : 256


,rating,rating_filtered,item_nbr,item,option,date,name,full_review,pos,pos_filtered
251,5,1,1168550,삼성 외장하드 J3,500GB,2016.01.22,조*식,. ㅎ,"[(., SF), (ㅎ, NNG)]",[ㅎ]
252,4,1,1168550,삼성 외장하드 J3,1TB,2016.01.19,김*진,. 삼성은 배신하지 않습니다 ㅎ 잘쓰겠습니다.,"[(., SF), (삼성, NNG), (은, JX), (배신, NNG), (하, X...","[삼성, 배신, ㅎ, 잘]"
253,5,1,1168550,삼성전자 J3 Portable 외장하드,2TB,2016.01.19,이*호,. 배송 굿~ 아직 사용은 안했네요,"[(., SF), (배송, NNG), (굿, NNG), (~, SO), (아직, M...","[배송, 굿, 아직, 사용]"
254,5,1,1168550,삼성전자 J3 Portable 외장하드,2TB,2015.12.02,이*훈,. 디자인도 세련되고 성능도 조아요~~,"[(., SF), (디자인, NNG), (도, JX), (세련, NNG), (되, ...","[디자인, 세련, 고, 성능, 조아]"
255,4,1,1168550,삼성전자 J3 Portable 외장하드,2TB,2015.11.30,이*민,. 전용 케이스가 없는게 다소 아쉽네요.,"[(., SF), (전용, NNG), (케이스, NNG), (가, JKS), (없,...","[전용, 케이스, 없, 다소, 아쉽]"


In [112]:
X_test = Vectorizer(df_train, 'pos_filtered', df_test)
y_test = df_test['rating_filtered']

In [113]:
X_test.tail()

,0,1,2,3,4,5,6,7,8,9,...,4410,4411,4412,4413,4414,4415,4416,4417,4418,4419
251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
252,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
253,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
# model = MultinomialNB()
# model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

          0       0.62      0.40      0.48        20
          1       0.95      0.98      0.96       236

avg / total       0.92      0.93      0.93       256



In [ ]:
멀티 스레드 처리
스크래피
안좋은 데이터 불리거나, 좋은 데이터 줄이거나

In [ ]:
# 현재까지의 모든 데이터로 모델링 후
# 기간지정 데이터를 test 데이터로 만들어서 예측
# 상품평마다 긍정 / 부정 예측 후 총 데이터에서 긍정 / 부정 데이터 비중 출력
# 별점 상관없이 상품명 키워드만 보고 긍정 / 부정 예측하는 것이 목표 -> 테스트 데이터 수동으로 라벨링 한 후 다시 report 출력
# 긍정 데이터 주요 키워드 출력, 부정 데이터 주요 키워드 출력
# * 긍정, 부정 데이터 비대칭 해결해야함
# 부정적이지 않은 단어들의 filtering(부분적으로 부정 데이터에만 stop word 적용?)
# 새로운 단어 -> missing value X, 중요한 단어일 수도 있는데 그냥 through된다는 것
# 품사필터링 + stop word 설정으로 모델 업데이트 필요

# countvector 파이프라인 X(함수로 만들어서 적용하면 됨), tf-idf(성능보고 선택적으로)
# kkma 사용자 사전 추가(자바 빌드)

In [ ]:
def Vectorizer(df_train, pos_filtered_col, df_input):
    """
    df_fin의 'pos_filtered' column을 받아 vectorized df를 생성해주는 함수
    """
    train_corpus = []
    for i in range(len(df_train[pos_filtered_col])):
        for ii in range(len(df_train[pos_filtered_col][i])):
            train_corpus.append(df_train[pos_filtered_col][i][ii])
            
    from sklearn.feature_extraction.text import CountVectorizer
    vect = CountVectorizer(token_pattern=r"\b\w+\b") # 한 글자도 corpus에 포함될 수 있게 해주는 정규표현식
    vect.fit(train_corpus)

    vect_ls = []
    for i in range(len(df_input)):
        vect_ls.append((vect.transform(df_input[pos_filtered_col][i]).toarray().sum(axis=0).tolist()))
    
    df_vec = pd.DataFrame(vect_ls)
    return df_vec

In [ ]:
휴리스틱 튜닝
가짜 데이터 추가
stop word도 수동으로
나이브베이즈 parameter - sample weight '으나'

In [40]:
df_test_['pos'].iloc[15]

[('.', 'SF'),
 ('예쁘', 'VA'),
 ('어서', 'ECD'),
 ('구매', 'NNG'),
 ('하', 'XSV'),
 ('었', 'EPT'),
 ('으나', 'ECE'),
 (',', 'SP'),
 ('바닥', 'NNG'),
 ('이', 'JKS'),
 ('비스듬히', 'MAG'),
 ('되', 'VV'),
 ('어서', 'ECD'),
 ('스', 'VV'),
 ('어', 'ECS'),
 ('있', 'VXV'),
 ('기', 'ETN'),
 ('가', 'JKS'),
 ('위태', 'XR'),
 ('롭', 'XSA'),
 ('고', 'ECE'),
 ('자꾸', 'MAG'),
 ('넘어지', 'VV'),
 ('ㅂ니다', 'EFN'),
 ('.', 'SF'),
 ('프', 'UN'),
 ('이', 'VCP'),
 ('라', 'EFN'),
 ('스틱', 'NNG'),
 ('재질', 'NNG'),
 ('이', 'VCP'),
 ('라', 'ECD'),
 ('고급', 'NNG'),
 ('스럽', 'XSA'),
 ('ㅁ', 'ETN'),
 ('과', 'JKM'),
 ('는', 'JX'),
 ('전혀', 'MAG'),
 ('거리', 'NNG'),
 ('가', 'JKS'),
 ('멀', 'VV'),
 ('며', 'ECE'),
 (',', 'SP'),
 ('조잡', 'XR'),
 ('하', 'XSA'),
 ('ㄴ', 'ETD'),
 ('장난감', 'NNG'),
 ('수준', 'NNG'),
 ('의', 'JKG'),
 ('느낌', 'NNG'),
 ('이', 'VCP'),
 ('ㅂ니다', 'EFN'),
 ('.', 'SF'),
 ('가격', 'NNG'),
 ('적', 'XSN'),
 ('으로', 'JKM'),
 ('고급', 'NNG'),
 ('적', 'XSN'),
 ('이', 'VCP'),
 ('ㄴ', 'ETD'),
 ('느낌', 'NNG'),
 ('을', 'JKO'),
 ('바라', 'VV'),
 ('ㄴ', 'ETD'),
 ('것', 'NNB'),
 ('은', 

In [ ]:
kkma-2.0.jar

In [28]:
import requests
from scrapy.http import TextResponse
import json
search_url = "http://www.coupang.com/np/search?component=&q=조명"
rep = requests.get(search_url)
response = TextResponse(rep.url, body=rep.text, encoding='utf-8')

# 상품마다의 링크를 불러오기에 앞서 아이템 Top 10 리스트 불러오기
products = json.loads(response.xpath('//*[@id="productList"]/@data-products').extract()[0])['indexes'][:5]

df = pd.DataFrame(columns=['rating', "item_nbr", "item", "option", "date", "name", "re_title", 'review']) 

review_urls = {}

for product in products:

    # Top 10 상품마다의 상품평 개수(나중에 불러오는 정보라 상품링크 안에서는 xpath로 바로 가져올 수 없음)
    try:
#         review_counts[product] = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
        review_count = int(response.xpath('//*[@id="{}"]/a/dl/dd/div/div[4]/div[2]/span[2]/text()'.format(product)).extract()[0].strip("()"))
    except Exception as e:
        print('No reviews')

    def make_url(product, review_count):
        page_count = (review_count//100)+1
        review_urls = []
        for page in range(1, page_count+1):
            review_urls.append('http://www.coupang.com/vp/product/reviews?productId={}&page={}&size=100&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=2&ratingSummary=true'.format(product, page))
            # size는 100이 최대로 설정되어 있음
        return review_urls


    # Top 10 상품마다의 크롤링할 url(리뷰 크롤링은 '상품평 보기' 클릭 후 나타나는 reviews Request URL에서 해야함)
    review_urls[product] = make_url(product, review_count)
    review_url = make_url(product, review_count) # 리스트로 반환됨
#     print(review_url)

    for url in review_url:

        review_url = url
        review_rep = requests.get(review_url)
        review_response = TextResponse(review_rep.url, body=review_rep.text, encoding='utf-8')        

        # Top 10 상품마다의 상품평 개수
        review_total_cnt = int(review_response.xpath('/html/body/div[2]/@data-review-total-count').extract()[0])

        # Top 10 상품마다 한 page의 상품평 개수
        review_url_cnt = int(float(review_response.xpath('count(/html/body/article)').extract()[0]))

        if review_url_cnt < 100:          
            for i in range(1, review_url_cnt+1): # 한개의 review_url에서 100개를 훑고 그 다음 review_url로 넘어가면 됨
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}

        else:
            for i in range(1, 101):
                item_nbr = product

                date = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[2]/text()'.format(i)).extract()[0]

                name = review_response.xpath('/html/body/article[{}]/div[1]/div[2]/span/text()'.format(i)).extract()[0][:-1]

                item_option_ls = review_response.xpath('/html/body/article[{}]/div[1]/div[4]/text()'.format(i)).extract()[0].split(",")
                if len(item_option_ls) == 1:
                    item = item_option_ls[0]

                else:    
                    item = item_option_ls[0]
                    option = item_option_ls[1]

                if review_response.xpath('/html/body/article[{}]/div[3]/@class'.format(i)).extract()[0] == 'sdp-review__article__list__headline':
                    re_title = review_response.xpath('/html/body/article[{}]/div[3]/text()'.format(i)).extract()
                    review = review_response.xpath('/html/body/article[{}]/div[4]/div/text()'.format(i)).extract()

                else:
                    re_title = ['.']
                    review = review_response.xpath('/html/body/article[{}]/div[3]/div/text()'.format(i)).extract()

                rating = review_response.xpath('/html/body/article[{}]/div[1]/div[3]/div[1]/div/@data-rating'.format(i)).extract()[0]

                df.loc[len(df)] = {'rating':rating, 'item_nbr':item_nbr, 'item':item, 'option':option, 'date':date, 'name':name, 're_title':re_title, 'review':review}

In [29]:
len(df)

5873

In [30]:
df.tail()

,rating,item_nbr,item,option,date,name,re_title,review
5868,3,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.12.26,박*화,[.],[]
5869,5,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.12.26,차*지,[.],[]
5870,5,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.12.20,민또,[.],[]
5871,5,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.12.16,정*현,[.],[]
5872,5,36815948,본진 판타스틱 스타 줄조명 20구,One Color,2017.11.16,강*희,[.],[]


In [78]:
def re_title_filter(a):
    for i in range(len(a[0].split('\n'))):
        if len(a[0].split('\n')[i].strip()) != 0:
            return a[0].split('\n')[i].strip()
        else:
            return '.'

def review_filter(a):
    s = ''
    for i in range(len(a)):
        for ii in range(len(a[i].split('\n'))):
            if a[i].split('\n')[ii].strip() != 0:
                s += a[i].split('\n')[ii].strip() + ' '
    return s

def kkma_pos(a):
    from konlpy.tag import Kkma
    kkma = Kkma()
    return kkma.pos(a)

def kkma_pos_filter(a):
    """
    df['pos']를 받아 품사 필터링을 거쳐 해당 키워드 리스트 반환
    """
    ls = []
    for i in range(len(a)):
        #NNG(보통 명사), NNP(고유 명사), VA(형용사), XR(어근), MAG(일반 부사), UN(명사추정범주)
        if a[i][1]=='NNG' or a[i][1]=='NNP' or a[i][1]=='VA' or a[i][1]=='UN' or a[i][1]=='XR' or a[i][1]=='MAG':
            ls.append(a[i][0])
    return ls

def rating_filter(a):
    if a == '4' or a == '5':
        return 1
        # 긍정
    else:
        return 0
        # 부정

In [79]:
df_ = df[253:255]
df_['re_title_filtered'] = df_['re_title'].apply(re_title_filter)
df_['review_filtered'] = df_['review'].apply(review_filter)
df_['full_review'] = df_['re_title_filtered']+df_['review_filtered']
df_['pos'] = df_['full_review'].apply(kkma_pos)
df_['pos_filtered'] = df_['pos'].apply(kkma_pos_filter)
df_['rating_filtered'] = df_['rating'].apply(rating_filter)

df_.tail(3)
# df_fin = df_[['rating', 'rating_filtered', 'item_nbr', 'item', 'option', 'date', 'name', 'full_review', 'pos', 'pos_filtered']]
# df_fin.tail(3)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,rating,item_nbr,item,option,date,name,re_title,review,re_title_filtered,review_filtered,full_review,pos,pos_filtered,rating_filtered
253,5,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2018.04.11,박*기,[\n \n \n ...,[\n \n ...,.,너무 이뻐요ㅠㅠㅠㅠ진짜 사길잘했다는 생각 가격도 저렴하고 혼자서도 척척 잘조립되...,. 너무 이뻐요ㅠㅠㅠㅠ진짜 사길잘했다는 생각 가격도 저렴하고 혼자서도 척척 잘조립...,"[(., SF), (너무, MAG), (이쁘, VA), (어요, EFN), (ㅠㅠㅠ...","[너무, 이쁘, 진짜, 길, 생각, 가격, 저렴, 혼자, 척척, 잘, 조립, 그냥,...",1
254,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2018.02.09,박*서,[\n \n 배송상태 ...,[\n \n ...,.,구멍이란 구멍엔 전부 스트로폼이 잔뜩 꼈군요. 제거하느라 면봉이며 이쑤시개며 고...,. 구멍이란 구멍엔 전부 스트로폼이 잔뜩 꼈군요. 제거하느라 면봉이며 이쑤시개며 ...,"[(., SF), (구멍, NNG), (이, VCP), (란, ETD), (구멍, ...","[구멍, 구멍, 전부, 스트로, 폼, 잔뜩, 제거, 면봉, 이쑤시개, 고생, 평, ...",0


In [61]:
df_

,rating,item_nbr,item,option,date,name,re_title,review,re_title_filtered,review_filtered,full_review
253,5,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2018.04.11,박*기,[\n \n \n ...,[\n \n ...,None,너무 이뻐요ㅠㅠㅠㅠ진짜 사길잘했다는 생각 가격도 저렴하고 혼자서도 척척 잘조립되...,NaN
254,1,1001428,스피아노 스프링 플로어 장스탠드 SS-2000A,블랙,2018.02.09,박*서,[\n \n 배송상태 ...,[\n \n ...,"배송상태 불량 200%, 제품상태 -200%",구멍이란 구멍엔 전부 스트로폼이 잔뜩 꼈군요. 제거하느라 면봉이며 이쑤시개며 고...,"배송상태 불량 200%, 제품상태 -200% 구멍이란 구멍엔 전부 스트로폼이 잔뜩..."


In [62]:
df_['re_title']

253    [\n                \n                     \n  ...
254    [\n                \n                    배송상태 ...
Name: re_title, dtype: object

In [77]:
df_['re_title'][254][0].split('\n')

for i in range(len(df_['re_title'][254][0].split('\n'))):
    if len(df_['re_title'][254][0].split('\n')[i].strip()) != 0:
        print(df_['re_title'][254][0].split('\n')[i].strip(), type(df_['re_title'][254][0].split('\n')[i].strip()))
    else:
        print('none')

none
none
배송상태 불량 200%, 제품상태 -200% <class 'str'>
none
none


In [ ]:
def re_title_filter(a):
    for i in range(len(a[0].split('\n'))):
        if len(a[0].split('\n')[i].strip()) != 0:
            return a[0].split('\n')[i].strip()